# Data

In [ ]:
from pathlib import Path

data_dir = Path("/home/jasseur/Downloads")

In [ ]:
!wget -O $data_dir/MOT15.zip "https://motchallenge.net/data/MOT15.zip"
!wget -O $data_dir/MOT16.zip "https://motchallenge.net/data/MOT16.zip"
!wget -O $data_dir/MOT17.zip "https://motchallenge.net/data/MOT17.zip"

In [ ]:
!unzip $data_dir/MOT15.zip -d $data_dir/
!unzip $data_dir/MOT16.zip -d $data_dir/MOT16/
!unzip $data_dir/MOT17.zip -d $data_dir/

In [ ]:
!rm $data_dir/MOT15.zip
!rm $data_dir/MOT16.zip
!rm $data_dir/MOT17.zip

In [ ]:
!ls $data_dir/MOT15

In [ ]:
train_mot15_dir = Path(data_dir, "MOT15", "train")

In [ ]:
!ls $train_mot15_dir

In [ ]:
!ls $train_mot15_dir/ADL-Rundle-6

In [ ]:
!cat $train_mot15_dir/ADL-Rundle-6/seqinfo.ini

In [ ]:
import configparser
import pandas as pd

def get_labels(data_dir):
    gt_dfs = []
    read_config = configparser.ConfigParser()
    for dir_path in data_dir.iterdir():
        read_config.read(Path(dir_path, "seqinfo.ini"))
        name = read_config.get("Sequence", "name")
        frameRate = read_config.get("Sequence", "frameRate")
        seqLength = read_config.get("Sequence", "seqLength")
        gt_path = Path(dir_path, "gt/gt.txt")
        gt_header_list = ["frame_id", "person_id", "x", "y", "w", "h", "cls_id"]
        gt_df = pd.read_csv(gt_path, names=gt_header_list, usecols=[0, 1, 2, 3, 4, 5, 6])
        gt_df["video_name"] = name
        gt_df["video_length"] = seqLength
        gt_df["time"] = gt_df["frame_id"].apply(lambda x: (x - 1) / int(frameRate))
        gt_dfs.append(gt_df)
    gt_df = pd.concat(gt_dfs, ignore_index=True)
    return gt_df

train_mot15_dir = Path(data_dir, "MOT15", "train")
train_mot16_dir = Path(data_dir, "MOT16", "train")
train_mot17_dir = Path(data_dir, "MOT17", "train")
train_mot15_df = get_labels(train_mot15_dir)
train_mot16_df = get_labels(train_mot16_dir)
train_mot17_df = get_labels(train_mot17_dir)

In [ ]:
train_mot17_df.head()

In [ ]:
train_mot15_df.to_csv(Path("mot15", "annotation_train.csv"), index=False)
train_mot16_df.to_csv(Path("mot16", "annotation_train.csv"), index=False)
train_mot17_df.to_csv(Path("mot17", "annotation_train.csv"), index=False)